Data Collection

In [11]:
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm

# importing pytorch libraries
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.models as models
import torchvision.transforms as transforms

# importing image required libraries
import time
import os
import PIL.Image as Image
from IPython.display import display

# importing sklearn libraries for evaluation
from sklearn.metrics import confusion_matrix, classification_report

cpu


In [48]:
# Count all img files of all car classes
local_path = '/Users/minhnhat/CarTypeClassification/archive/car_data/car_data'
train_path = os.path.join(local_path, 'train')
test_path = os.path.join(local_path, 'test')

car_count_dict = {}
for folder_name in os.listdir(train_path):
    count = len(os.listdir(os.path.join(train_path,folder_name)))
    car_count_dict[folder_name] = count

for folder_name in os.listdir(test_path):
    # skip the invisible file
    if folder_name == ".DS_Store":
        continue
    count = len(os.listdir(os.path.join(test_path,folder_name)))
    car_count_dict[folder_name] += count

print(len(car_count_dict))
print(car_count_dict)

196
{'Dodge Dakota Crew Cab 2010': 82, 'GMC Canyon Extended Cab 2012': 80, 'Hyundai Santa Fe SUV 2012': 84, 'Dodge Durango SUV 2012': 87, 'Chevrolet Silverado 1500 Extended Cab 2012': 87, 'Ford Mustang Convertible 2007': 89, 'Buick Rainier SUV 2007': 85, 'Jeep Compass SUV 2012': 85, 'Chevrolet Silverado 2500HD Regular Cab 2012': 76, 'Ford Edge SUV 2012': 86, 'Dodge Ram Pickup 3500 Quad Cab 2009': 88, 'Bentley Continental Supersports Conv. Convertible 2012': 73, 'Land Rover Range Rover SUV 2012': 85, 'Hyundai Tucson SUV 2012': 87, 'Honda Odyssey Minivan 2007': 82, 'Audi 100 Wagon 1994': 85, 'Dodge Challenger SRT8 2011': 78, 'Mercedes-Benz Sprinter Van 2012': 82, 'Bentley Continental GT Coupe 2012': 69, 'Rolls-Royce Ghost Sedan 2012': 77, 'Lamborghini Gallardo LP 570-4 Superleggera 2012': 71, 'Dodge Charger SRT-8 2009': 84, 'Volkswagen Beetle Hatchback 2012': 85, 'Audi TT RS Coupe 2012': 79, 'Acura TSX Sedan 2012': 81, 'Jaguar XK XKR 2012': 93, 'Aston Martin V8 Vantage Convertible 2012':

Data Preprocessing

In [52]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
# ImageDataGenerator for data augmentation
train_datagen = ImageDataGenerator(rescale=1./255, rotation_range=20, width_shift_range=0.2,
                                   height_shift_range=0.2, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_path, target_size=(224, 224), batch_size=32, class_mode='categorical')
val_generator = test_datagen.flow_from_directory(test_path, target_size=(224, 224), batch_size=32, class_mode='categorical')

Found 8144 images belonging to 196 classes.
Found 8041 images belonging to 196 classes.


In [53]:
from tensorflow.keras.applications import MobileNetV3Large
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D

base_model = MobileNetV3Large(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(196, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


12683000/12683000 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


NameError: name 'num_classes' is not defined